In [9]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

import json
import requests

In [18]:
KAKAO_TOKEN_SAVED_FILE = "res/kakao_msg/kakao_token.json"
APP_KEY_SAVED_FILE = "res/kakao_msg/app_key"

In [16]:
def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)

In [17]:
# 초기 한 번만 수행함(REST APP KEY 저장 목적)
import pickle

app_key = ""

f = open(APP_KEY_SAVED_FILE, 'wb')
pickle.dump(app_key, f)
f.close()

In [20]:
# REST APP KEY 저장 잘 되었는지 확인하는 목적
f = open(APP_KEY_SAVED_FILE, 'rb')
app_key = pickle.load(f)
f.close()

print(app_key)

In [11]:
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

In [28]:
def update_tokens(app_key, filename):
    tokens = load_tokens(filename)
    
    url = "https://kauth.kakao.com/oauth/token"
    
    data = {
        "grant_type" : "refresh_token",
        "client_id" : f"{app_key}",
        "refresh_token" : tokens['refresh_token']
    }
    
    print("data: ", data)
    
    response = requests.post(url, data=data)
    
    if response.status_code != 200:
        print("Error!")
        print("status: ", response.status_code)
        print("res: ", response.json())
        tokens = None
    else:
        print(response.json())
        
        # 기존 내용 백업
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = filename + "." + now
        os.rename(filename, backup_filename)
        
        # 새롭게 업데이트된 토큰 저장
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)
        
    return tokens

In [ ]:
#    GET /oauth/authorize?client_id={REST_API_KEY}&redirect_uri={REDIRECT_URI}&response_type=code HTTP/1.1
#    Host: kauth.kakao.com

In [42]:
# 구동시키지 않아도됨
def get_auth_code():
    url = "https://kauth.kakao.com/oauth/authorize"
    
    response = requests.get(url, params = {'client_id': app_key, 
                                           'redirect_uri': 'https://localhost.com', 
                                           'response_type': 'code'})
    print(response)

In [54]:
# 재발급이 필요한 경우 기존 토큰 발급 절차를 그대로 재활용
url = "https://kauth.kakao.com/oauth/token"

data = {
    "grant_type" : "authorization_code",
    "client_id" : app_key,
    "redirect_uri" : "https://localhost.com",
    "code" : ""
}

response = requests.post(url, data=data)

if response.status_code != 200:
    print("Error!")
else:
    tokens = response.json()
    print(tokens)

In [45]:
# 토큰을 다시 저장
save_tokens(KAKAO_TOKEN_SAVED_FILE, tokens)

In [43]:
# 안돌려도 됨
get_auth_code()

<Response [200]>


In [ ]:
app = Flask(__name__)
CORS(app)

@app.route('/spring2pythonRequestWithData', methods = ['GET', 'POST'])
def spring2pythonRequestWithData():
    
    params = json.loads(request.get_data(), encoding='utf-8')
    print("params: ", params)
    
    if len(params) == 0:
        return jsonify("No Parameter")
    
    params_string = ""
    customer_name = ""
    price = ""
    
    ##################### 개조 구간 시작 #####################
    #params_string = ""
    #for key in params.keys():
    #    params_string += 'key: {}, value: {}<br>'.format(key, params[key])
    
    values = []
    for key in params.keys():
        values.append(params[key])

    print("values: ", values)
    customer_name = values[0]
    price = values[1]
    
    print("이름: ", customer_name)
    print("가격: ", price)
    
    tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)
    #print("tokens: ", tokens)

    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

    headers = {
        "Authorization" : "Bearer " + tokens['access_token']
    }
    
    message = "주문 테스트 - 구매자: {0}, 결제 금액: {1}".format(customer_name, price)

    data = {
        "template_object" : json.dumps({
            "object_type" : "text",
            "text" : message,
            "link" : {
                "web_url" : "http://localhost:8080/"
            }
        })
    }

    response = requests.post(url, headers=headers, data=data)
    print(response.status_code)
    
    if response.status_code == 401:
        tokens = update_tokens(app_key, KAKAO_TOKEN_SAVED_FILE)
        
    #print("tokens: ", tokens)
    
    ##################### 개조 구간 끄읕 #####################
    
    return jsonify(params_string)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
<ipython-input-55-b2d64cbfd9ff>:7: DeprecationWarning: 'encoding' is ignored and deprecated. It will be removed in Python 3.9
  params = json.loads(request.get_data(), encoding='utf-8')
127.0.0.1 - - [05/Oct/2021 19:27:31] "POST /spring2pythonRequestWithData HTTP/1.1" 200 -


params:  {'name': '결제테스트', 'cost': 1000000000}
values:  ['결제테스트', 1000000000]
이름:  결제테스트
가격:  1000000000
200
